In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
root_dir = "./generate_csv/"

def get_file_by_prefix(root_dir,prefix):
    all_file = os.listdir(root_dir)
    results = []
    for filename in all_file:
        if filename.startswith(prefix):
            results.append(os.path.join(root_dir,filename))
    return results

train_filenames = get_file_by_prefix(root_dir,"train")
valid_filenames = get_file_by_prefix(root_dir,"valid")
test_filenames = get_file_by_prefix(root_dir,"test")
import pprint 
pprint.pprint (train_filenames)

['./generate_csv/train_15.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_00.csv',
 './generate_csv/train_14.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_16.csv',
 './generate_csv/train_17.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_13.csv',
 './generate_csv/train_12.csv',
 './generate_csv/train_06.csv',
 './generate_csv/train_10.csv',
 './generate_csv/train_04.csv',
 './generate_csv/train_05.csv',
 './generate_csv/train_11.csv',
 './generate_csv/train_08.csv',
 './generate_csv/train_09.csv',
 './generate_csv/train_19.csv',
 './generate_csv/train_18.csv']


In [17]:
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)]*n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1]) #tf.stack:转化为向量
    y = tf.stack(parsed_fields[-1:])
    return x,y

def csv_reader_dataset(filenames, n_readers=5,
                      batch_size=32, n_parse_treads=5,
                      shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat() #重复无限次,
    #转化为文本内容
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size) #数据进行混排，shuffle_buffer_size内容的大小
    #解析的工作
    dataset = dataset.map(parse_csv_line,
                         num_parallel_calls=n_parse_treads)
    dataset = dataset.batch(batch_size)   
    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames,batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames,batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames,batch_size=batch_size)

In [21]:
def serialize_example(x, y):
    input_features = tf.train.FloatList(value = x)
    label = tf.train.FloatList(value = y)
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(
            float_list = input_features),
            "label":tf.train.Feature(float_list = label)
        }
    )
    example = tf.train.Example(features = features)
    return example.SerializeToString()

def csv_dataset_to_tfrecords(base_filename, dataset,
                             n_shards, steps_per_shard,
                             compression_type = None):
    all_filenames=[]
    options = tf.io.TFRecordOptions(
        compression_type = compression_type)
    for shard_id in range(n_shards):
        filename_fullpath = "{}_{:05d}_of_{:05d}".format(base_filename,
                                                        shard_id,n_shards)
        with tf.io.TFRecordWriter(filename_fullpath,options) as writer:
            for x_batch, y_batch in dataset.take(steps_per_shard):
                for x_example, y_example in zip(x_batch,y_batch):
                    writer.write(
                    serialize_example(x_example,y_example))
        all_filenames.append(filename_fullpath)
    return all_filenames

In [22]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir,"train")
valid_basename = os.path.join(output_dir,"vlaid")
test_basename = os.path.join(output_dir,"test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard)
test_tfrecord_filenames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard)


In [23]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "generate_tfrecords_gzip"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir,"train")
valid_basename = os.path.join(output_dir,"vlaid")
test_basename = os.path.join(output_dir,"test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard,
    compression_type="GZIP")
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard,
    compression_type="GZIP")
test_tfrecord_filenames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard,
    compression_type="GZIP")

In [24]:
pprint.pprint(train_tfrecord_filenames)
pprint.pprint(valid_tfrecord_filenames)
pprint.pprint(test_tfrecord_filenames)

['generate_tfrecords_gzip/train_00000_of_00020',
 'generate_tfrecords_gzip/train_00001_of_00020',
 'generate_tfrecords_gzip/train_00002_of_00020',
 'generate_tfrecords_gzip/train_00003_of_00020',
 'generate_tfrecords_gzip/train_00004_of_00020',
 'generate_tfrecords_gzip/train_00005_of_00020',
 'generate_tfrecords_gzip/train_00006_of_00020',
 'generate_tfrecords_gzip/train_00007_of_00020',
 'generate_tfrecords_gzip/train_00008_of_00020',
 'generate_tfrecords_gzip/train_00009_of_00020',
 'generate_tfrecords_gzip/train_00010_of_00020',
 'generate_tfrecords_gzip/train_00011_of_00020',
 'generate_tfrecords_gzip/train_00012_of_00020',
 'generate_tfrecords_gzip/train_00013_of_00020',
 'generate_tfrecords_gzip/train_00014_of_00020',
 'generate_tfrecords_gzip/train_00015_of_00020',
 'generate_tfrecords_gzip/train_00016_of_00020',
 'generate_tfrecords_gzip/train_00017_of_00020',
 'generate_tfrecords_gzip/train_00018_of_00020',
 'generate_tfrecords_gzip/train_00019_of_00020']
['generate_tfrecords

In [42]:
expected_features = {
    "input_features":tf.io.FixedLenFeature([8],dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1],dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"],example["label"]


def tfrecords_reader_dataset(filenames, n_readers=5,
                             batch_size=32, n_parse_threads=5,
                             shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TFRecordDataset(
            filename, compression_type = "GZIP"),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames,
                                           batch_size = 3)
for x_batch, y_batch in tfrecords_train.take(2):
    print(x_batch)
    print(y_batch)

tf.Tensor(
[[ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[2.419]
 [2.419]
 [2.419]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[2.419]
 [2.419]
 [1.832]], shape=(3, 1), dtype=float32)


In [39]:
batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(
    train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(
    valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(
    test_tfrecord_filenames, batch_size = batch_size)

In [40]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

history = model.fit(tfrecords_train_set,
                    validation_data = tfrecords_valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Epoch 1/100
348/348 [==============================] - 2s 5ms/step - loss: 0.8213 - val_loss: 0.7628
Epoch 2/100
348/348 [==============================] - 1s 4ms/step - loss: 0.4865 - val_loss: 0.6811
Epoch 3/100
348/348 [==============================] - 1s 3ms/step - loss: 0.4318 - val_loss: 0.7125
Epoch 4/100
348/348 [==============================] - 1s 4ms/step - loss: 0.3983 - val_loss: 0.6786
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3823 - val_loss: 0.4614
Epoch 6/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3739 - val_loss: 0.5026
Epoch 7/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3611 - val_loss: 0.5732
Epoch 8/100
348/348 [==============================] - 2s 4ms/step - loss: 0.3503 - val_loss: 0.5546
Epoch 9/100
348/348 [==============================] - 2s 4ms/step - loss: 0.3395 - val_loss: 0.4723
Epoch 10/100
348/348 [==============================] - 2s 5ms/step - loss: 0.3365 - val_lo